In [1]:
# Lab 4 - Python Version

# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import statsmodels.api as sm
import matplotlib.pyplot as plt

/Users/mateorobbins/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/mateorobbins/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


ImportError: cannot import name '_centered' from 'scipy.signal.signaltools' (/Users/mateorobbins/opt/anaconda3/lib/python3.9/site-packages/scipy/signal/signaltools.py)

## Lab 4: Fire and Tree Mortality

The database we'll be working with today includes 36066 observations of individual trees involved in prescribed fires and wildfires occurring over 35 years, from 1981 to 2016. It is a subset of a larger fire and tree mortality database from the US Forest Service (see data description for the full database here: [link](https://www.nature.com/articles/s41597-020-0522-7#Sec10)). Our goal today is to predict the likelihood of tree mortality after a fire.

In [2]:
# Data Exploration
# Load the dataset
trees_dat = pd.read_csv("https://raw.githubusercontent.com/MaRo406/eds-232-machine-learning/main/data/trees-dat.csv")

# Quick overview of the data
print(trees_dat.head())
print(trees_dat.info())

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Question 1: Recode predictors to zero-based integer form if categorical
for col in ['YrFireName', 'Species', 'Genus_species']:
    trees_dat[col] = trees_dat[col].astype('category').cat.codes

In [ ]:
# Question 2: Data Splitting (70% training, 30% test)
trees_training, trees_test = train_test_split(trees_dat, test_size=0.3, random_state=42)

# Question 3: Number of training observations
print("Training set observations:", trees_training.shape[0])

# Question 4: Correlation check for selecting top predictors
correlations = trees_training.corr()['yr1status'].sort_values(ascending=False)
top_predictors = correlations.index[1:4]
print("Top predictors:", top_predictors)

# Question 5: Simple Logistic Regression for top predictors
for predictor in top_predictors:
    X = trees_training[[predictor]]
    y = trees_training['yr1status']
    logit_model = sm.Logit(y, sm.add_constant(X)).fit()
    print(f"Logistic regression for predictor {predictor}:\n", logit_model.summary())

# Question 6: Interpreting Coefficients - Output given in summaries above

# Question 7: Visualize model fits
for predictor in top_predictors:
    plt.figure()
    plt.scatter(trees_training[predictor], trees_training['yr1status'], label='Data', alpha=0.5)
    plt.xlabel(predictor)
    plt.ylabel('yr1status')
    plt.title(f'Logistic Fit - Predictor: {predictor}')
    plt.show()

# Question 8: Multiple Logistic Regression with all predictors
X_full = trees_training[top_predictors]
y_full = trees_training['yr1status']
logistic_full = sm.Logit(y_full, sm.add_constant(X_full)).fit()
print("Multiple logistic regression model:\n", logistic_full.summary())

# Question 9: Cross-validation for model accuracy
kf = KFold(n_splits=10, shuffle=True, random_state=42)
models = [LogisticRegression() for _ in range(4)]  # Models for each top predictor and full model

for i, predictor in enumerate(list(top_predictors) + [top_predictors]):
    X = trees_training[[predictor]] if isinstance(predictor, str) else trees_training[top_predictors]
    y = trees_training['yr1status']
    accuracy = cross_val_score(models[i], X, y, cv=kf, scoring='accuracy').mean()
    print(f"Cross-validated accuracy for model {i+1} ({predictor}): {accuracy}")

# Question 10: Highest accuracy identified in previous step

# Question 11: Compute confusion matrix on most accurate model
X_best = trees_training[top_predictors]  # assuming this is the best model
y_best = trees_training['yr1status']
model_best = LogisticRegression().fit(X_best, y_best)
y_pred = model_best.predict(X_best)
conf_matrix = confusion_matrix(y_best, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Question 12: Explain confusion matrix - Typically provides counts of TP, FP, TN, FN

# Question 13: Overall model accuracy
accuracy = accuracy_score(y_best, y_pred)
print("Model accuracy:", accuracy)

# Question 14: Test model on test data
X_test = trees_test[top_predictors]
y_test = trees_test['yr1status']
y_test_pred = model_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy:", test_accuracy)

# Question 15: Compare test accuracy to cross-validation accuracy
print(f"Cross-validation accuracy: {accuracy}, Test accuracy: {test_accuracy}")